In [14]:
import cv2
import numpy as np
import os

# Specify the absolute path to the model files
prototxt_path = 'deploy.prototxt.txt'
caffemodel_path = 'res10_300x300_ssd_iter_140000_fp16.caffemodel'

# Load the pre-trained model from the disk
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

In [15]:
# initialize the video stream and allow the cammera sensor to warmup
cap = cv2.VideoCapture(0)

count = 0
save_path = r"C:\Users\dat20\Downloads"
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Prepare the frame to run through the deep learning model
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and predictions
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence < 0.5:
            continue

        # compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # draw the bounding box of the face along with the associated probability
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY),
            (0, 0, 255), 2)
        cv2.putText(frame, text, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

    # Save the captured image into the given directory
    cv2.imwrite(os.path.join(save_path, f"frame_{count}.jpg"), frame)
    count += 1

    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # if the `q` key is pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()